# Signal Editor

## Importando módulos

In [1]:
import os
import ipywidgets as widgets
from src.data.trunc_intervals import TruncIntervals
from src.data.signal_handler import Plotter

## Variáveis globais para o diretório dos dados e os canais

In [2]:
files_path = './dataset_eeg_cafe2022/formatted'
trunc_intervals_path = './assets/trunc_intervals'
channels = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

## Recuperando arquivos de sinal

In [3]:
files = [file for file in os.listdir(files_path) if '.csv' in file]

if len(files) == 0:
    print('Sem dados no diretório especificado.')
    exit()

## Criando flags para o estado dos widgets de arquivo e canal

In [4]:
file_widget_changed = False
channel_widget_changed = False

## Criando widget de output

In [5]:
output = widgets.Output()

## Criando instâncias das classes TruncIntervals e Plotter

In [6]:
trunc_intervals = TruncIntervals(trunc_intervals_path)
plotter = Plotter(files_path, output)

## Criando os widgets

#### Widgets para configurar o sinal

In [7]:
file_w = widgets.Combobox(options = files, value = files[0], description = 'Arquivo', layout = widgets.Layout(width = '300px'))
channel_w = widgets.Dropdown(options = channels, value = channels[0], description = 'Canal', layout = widgets.Layout(width = '300px'))
refresh_btn_w = widgets.Button(description = 'Refresh', layout = widgets.Layout(width = '70px'))
refresh_btn_w.layout.margin = '0px 0px 0px 50px'

signal_controller_hboxw = widgets.HBox([file_w, channel_w, refresh_btn_w])

#### Widgets para lidar com os intervalos de corte

In [8]:
label_w = widgets.Label(value = 'Intervalo de corte')
bottom_limit_w = widgets.FloatText(layout = widgets.Layout(width = '100px'))
top_limit_w = widgets.FloatText(layout = widgets.Layout(width = '100px'))
add_interval_btn_w = widgets.Button(description = 'Add', layout = widgets.Layout(width = '70px'))
pop_interval_btn_w = widgets.Button(description = 'Pop', layout = widgets.Layout(width = '70px'))
save_interval_btn_w = widgets.Button(description = 'Save', layout = widgets.Layout(width = '70px'))

trunc_intervals_hboxw = widgets.HBox([label_w, bottom_limit_w, top_limit_w, add_interval_btn_w, 
                                         pop_interval_btn_w, save_interval_btn_w], layout=widgets.Layout(justify_content='center'))

## Definindo eventos

#### Handlers para monitorar os widgets de arquivo e canal

In [9]:
def file_widget_change_handler(change):
    global file_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        file_widget_changed = True

In [10]:
def channel_widget_change_handler(change):
    global channel_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        channel_widget_changed = True 

#### Eventos de click

In [11]:
def refresh_button_clicked(btn):
    global file_widget_changed
    global channel_widget_changed
    if file_widget_changed:
        trunc_intervals.save_cutting_intervals()
        trunc_intervals.load_cutting_intervals(file_w.value)
        plotter.load_signal(file_w.value, channel_w.value)
        plotter.plot_signal(trunc_intervals.get_cutting_intervals(channel_w.value))
        file_widget_changed = False
        channel_widget_changed = False
    elif channel_widget_changed:
        plotter.change_current_fig(channel_w.value)
        plotter.plot_signal(trunc_intervals.get_cutting_intervals(channel_w.value))
        channel_widget_changed = False

In [12]:
def add_intervals_clicked(btn):
    start = bottom_limit_w.value
    end = top_limit_w.value
    trunc_intervals.add_interval(channel_w.value, start, end)
    plotter.plot_signal(trunc_intervals.get_cutting_intervals(channel_w.value))

In [13]:
def pop_intervals_clicked(btn):
    trunc_intervals.pop_interval(channel_w.value)
    plotter.plot_signal(trunc_intervals.get_cutting_intervals(channel_w.value))

In [14]:
def save_intervals_clicked(btn):
    trunc_intervals.save_cutting_intervals()

## Associando widgets aos seus eventos

In [15]:
file_w.observe(file_widget_change_handler)
channel_w.observe(channel_widget_change_handler)

refresh_btn_w.on_click(refresh_button_clicked)
add_interval_btn_w.on_click(add_intervals_clicked)
pop_interval_btn_w.on_click(pop_intervals_clicked)
save_interval_btn_w.on_click(save_intervals_clicked)

## Main

In [16]:
trunc_intervals.load_cutting_intervals(file_w.value)
plotter.load_signal(file_w.value, channel_w.value)
plotter.plot_signal(trunc_intervals.get_cutting_intervals(channel_w.value))

display(signal_controller_hboxw, output, trunc_intervals_hboxw)

Output()